La librería python-jobspy agrega resultados de LinkedIn, Indeed, Glassdoor y ZipRecruiter sin que tengas que configurar Selenium.

In [ ]:
#Librerías necesarias
import pandas as pd
from jobspy import scrape_jobs
import time
import random

In [ ]:
# 1. Configuración de la fragmentación
locations = ["Málaga, Spain", "Granada, Spain"]
jobs_presencial = []

print("Iniciando búsqueda fragmentada...")

for loc in locations:
    print(f"--- Extrayendo ofertas en: {loc} ---")
    
    try:
        jobs = scrape_jobs(
            site_name=["linkedin"],
            search_term="Data Engineer", # Cambia el puesto si es necesario
            location=loc,
            results_wanted=50,           # Cantidad moderada por ciudad
            is_remote=False,             # Prioriza presencial/híbrido
            linkedin_fetch_description=True 
        )
        
        # Convertir a DataFrame y añadir columna de origen para control
        df_loc = pd.DataFrame(jobs)
        if not df_loc.empty:
            df_loc['search_location'] = loc
            jobs_presencial.append(df_loc)
            print(f"Se han encontrado {len(df_loc)} ofertas en {loc}.")
        else:
            print(f"No se encontraron ofertas recientes en {loc}.")

    except Exception as e:
        print(f"Error al buscar en {loc}: {e}")

    wait_time = random.uniform(10, 20)
    print(f"Esperando {wait_time:.2f} segundos antes de la siguiente ciudad...\n")
    time.sleep(wait_time)


# 3. Consolidación de datos
if jobs_presencial:
    df_presencial = pd.concat(jobs_presencial, ignore_index=True)
    
    # Filtrar explícitamente por modalidad si el dataset trae la columna
    # (A veces LinkedIn etiqueta 'Hybrid' o 'On-site' en la descripción o metadatos)
    if 'job_type' in df_presencial.columns:
        # Aquí podrías filtrar si quisieras ser aún más estricto
        pass

    # Eliminar duplicados por URL (por si una oferta aparece en ambas ciudades)
    df_presencial = df_presencial.drop_duplicates(subset=['job_url'])
else:
    print("No se pudo recolectar ninguna oferta.")

Buscando ofertas... (esto puede tardar unos segundos)


2026-02-15 14:21:17,513 - INFO - JobSpy:Linkedin - finished scraping


¡Listo! Se han encontrado 30 ofertas únicas.


,title,company,location,date_posted,job_url
0,Data Engineer,Lighthouse,"Community of Madrid, Spain",2026-02-14,https://www.linkedin.com/jobs/view/4372989925
1,Data Engineer internship (m/f/x),Procter & Gamble España,"Madrid, Community of Madrid, Spain",2026-02-13,https://www.linkedin.com/jobs/view/4303479689
2,Junior Data Engineer,Refinery89,"Madrid, Community of Madrid, Spain",2026-02-13,https://www.linkedin.com/jobs/view/4372260104
3,Ingenieros/as de datos,ÁlamoConsulting,"Madrid, Community of Madrid, Spain",2026-02-13,https://www.linkedin.com/jobs/view/4364404433
4,Data Engineer PowerCenter,Logicalis Spain,"Madrid, Community of Madrid, Spain",2026-02-13,https://www.linkedin.com/jobs/view/4372006503


In [ ]:
# 1. Configuración para la búsqueda en remoto
print("--- Iniciando búsqueda de ofertas en REMOTO ---")

all_jobs = [df_presencial] if not df_presencial.empty else []

try:
    jobs_remote = scrape_jobs(
        site_name=["linkedin"],
        search_term="Data Engineer", # Mantén el mismo término para coherencia
        location="Spain",            # En remoto buscamos en todo el país
        results_wanted=50, 
        is_remote=True,              # FILTRO CLAVE: Solo remoto
        linkedin_fetch_description=True 
    )
    
    df_remote = pd.DataFrame(jobs_remote)
    
    if not df_remote.empty:
        df_remote['search_location'] = 'Remote (Spain)'
        all_jobs.append(df_remote)
        print(f"Se han encontrado {len(df_remote)} ofertas en remoto.")
    else:
        print("No se encontraron ofertas en remoto.")

except Exception as e:
    print(f"Error en la búsqueda remota: {e}")

# 2. Consolidación final (Igual que antes pero ahora incluye lo remoto)
if all_jobs:
    df_final = pd.concat(all_jobs, ignore_index=True)
    
    # Limpieza crucial: Eliminar duplicados
    # A veces una oferta de Málaga también se etiqueta como Remoto Nacional
    antes = len(df_final)
    df_final = df_final.drop_duplicates(subset=['job_url'])
    despues = len(df_final)
    
    print(f"\nProceso completado.")
    print(f"Total bruto: {antes} | Total tras limpiar duplicados: {despues}")

In [ ]:
from datetime import datetime

# 1. Generar el timestamp con formato: Año-Mes-Día_Hora-Minuto
# Ejemplo: 2026-02-15_14-30
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")

# 2. Definir el nombre del archivo usando f-string
nombre_archivo = f"ofertas_it_{timestamp}.csv"

# 3. Guardar el DataFrame
if not df_final.empty:
    df_final.to_csv(nombre_archivo, index=False, encoding='utf-8')
    print(f"--- Archivo guardado con éxito: {nombre_archivo} ---")
else:
    print("No hay datos para guardar.")